In [ ]:
!pip install transformers torch
!pip install langchain_community

In [2]:
from google.colab import userdata

key=userdata.get('HF_TOKEN')

In [8]:
from langchain.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token = key
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """
<|sys|>
        You are an expert in Web Scraping, so you are capable to find the information in HTML and label them accordingly. Please return the final result in JSON following the below format:
        product: Name of the product
        product_tag: CSS selectors or Xpaths associated with product
        price: Price of the product
        price_tag: CSS selectors or Xpaths associated with price
        desc: Description about the product
        desc_tag: CSS selectors or Xpaths associated with description of product
        img: Images associated with the product
        img_tag: CSS selectors or Xpaths
        </sys>
        <|user|>
        HTML: {html}
        </user>
        <|sys|>
        JSON:
        </sys>
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['html'], template='\n<|sys|>\nYou are an expert in Web Scraping, so you are capable to find the information in HTML and label them accordingly. Please return the final result in JSON.\n</sys>\n<|user|>     \nHTML: {html}\n</user>\n<|sys|>\nJSON: \n</sys>\n')

In [ ]:
conv = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

In [19]:
with open("test.html", 'r', encoding='utf-8') as file:
  html = file.read()

res = conv(
    {"html": f"{html}"}
)



> Entering new LLMChain chain...
Prompt after formatting:

<|sys|>
You are an expert in Web Scraping, so you are capable to find the information in HTML and label them accordingly. Please return the final result in JSON.
</sys>
<|user|>     
HTML: <div jscontroller="AtSb" class="w7Dbne CR1S4b" data-record-click-time="false" id="tsuid_48" jsdata="zt2wNd;_;A+m88g WDO8Ff;_;A+m88s" jsaction="rcuQ6b:npT2md;e3EWke:kN9HDb" data-hveid="CC4QAA">
    <div jsname="jXK9ad" class="uMdZh tIxNaf rllt__borderless" jsaction="mouseover:UI3Kjd;mouseleave:Tx5Rb;focusin:UI3Kjd;focusout:Tx5Rb">
      <div class="VkpGBb">
        <div class="cXedhc">
          <a class="vwVdIc wzN8Ac rllt__link a-no-hover-decoration" jsname="kj0dLd" data-cid="11265938073076301333" jsaction="click:h5M12e;" role="link" tabindex="0" data-ved="2ahUKEwjC7OW8romDAxXwFVkFHbmLAEoQ1YkKegQILhAB">
            <div>
              <div class="rllt__details">
                <div class="dbg0pd" aria-level="3" role="heading"><span class=

In [21]:
res['text']

'{\n  "name": "Houndstooth Coffee",\n  "rating": 4.6,\n  "reviews_count": 922,\n  "price_level": "$$",\n  "type": "Coffee shop",\n  "location": "401 Congress Ave. #100c",\n  "status": "Closed",\n  "open_time": "Opens 6:30 AM",\n  "description": "Cozy hangout for carefully sourced brews",\n  "img_url": "https://lh5.googleusercontent.com/p/AF1QipNRZ1ehiInk8CTrHCD08GnhcnF7e4q-1H8Qs8mG=w114-h114-n-k-no"\n}'

In [22]:
type(res['text'])

str

In [23]:
import json

js = json.dumps(res['text'])
js

'"{\\n  \\"name\\": \\"Houndstooth Coffee\\",\\n  \\"rating\\": 4.6,\\n  \\"reviews_count\\": 922,\\n  \\"price_level\\": \\"$$\\",\\n  \\"type\\": \\"Coffee shop\\",\\n  \\"location\\": \\"401 Congress Ave. #100c\\",\\n  \\"status\\": \\"Closed\\",\\n  \\"open_time\\": \\"Opens 6:30 AM\\",\\n  \\"description\\": \\"Cozy hangout for carefully sourced brews\\",\\n  \\"img_url\\": \\"https://lh5.googleusercontent.com/p/AF1QipNRZ1ehiInk8CTrHCD08GnhcnF7e4q-1H8Qs8mG=w114-h114-n-k-no\\"\\n}"'